# Integer Programming Homework

## Total: 40 Points

This notebook guides you through solving various Integer Programming (IP) problems and implementations. 
Follow the instructions provided in each section, write the required code, and answer the questions.

<em>NOTE: You should have the necessary setup from the LP Homework assignment.

# Question 1: Maya's Midnight March (10 points)
Maya's SCOPE team realized that they need someone to go out at midnight wearing white to test the visibility of pedestrians on crosswalks. Maya, who loves her hikes, happily volunteered. She needs to pack light in order to hit all the crosswalks in Needham. 

The maximum weight she wants to carry is 15 lbs. Here are the different items she can bring and their importance for her SCOPE team's data collection: 

+ Reflective Vest: Weight = 2 lbs, Importance = 6
+ Light Meter: Weight = 4 lbs, Importance = 12
+ DSLR Camera: Weight = 10 lbs, Importance = 4 (They really don't want to take pictures)
+ Notebook: Weight = 5 lbs, Importance = 8
+ Police Baton: Weight = 9 lbs, Importance = 10

### Formulate the IP
First, start by setting up the IP similar to the other problems. Write the objective function to maximize the total value of items brought.

Write the equations using LaTeX, between the $$ tags. Then convert the IP problem into code using this documentation: [PuLP Guide](https://realpython.com/linear-programming-python/#using-pulp). Using a binary variable ensures that only integer solutions are provided.

#### Solution:
Objective Function: $$ Z = 6x_1 + 12x_2 + 4x_3 + 8x_4 + 10x_5 $$ 
Constraint: $$ 2x_1 + 4x_2 + 10x_3 + 5x_4 + 9x_5 <= 15 $$ 

In [2]:
from pulp import *

# Define the Problem
model = LpProblem(name="mayas_march", sense=LpMaximize) # type: ignore

# Define the Variables
# TODO: Define each of the decision variables here (each item). Remember to assign them as binary
x1 = LpVariable(name="x1", cat="Binary")
x2 = LpVariable(name="x2", cat="Binary")
x3 = LpVariable(name="x3",cat="Binary")
x4 = LpVariable(name="x4", cat="Binary")
x5 = LpVariable(name="x5", cat="Binary")

# Objective Function
# TODO: Add objective function
model += 6*x1 + 12*x2 + 4*x3 + 8*x4 + 10*x5

# Constraints
# TODO: Add the weight constraint
model += 2*x1 + 4*x2 + 10*x3 + 5*x4 + 9*x5 <= 15

# Solve the Model
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/olga/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/47a8d1e4e64c4e49acbf6480a80f965e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/47a8d1e4e64c4e49acbf6480a80f965e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 27 RHS
At line 29 BOUNDS
At line 35 ENDATA
Problem MODEL has 1 rows, 5 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 30.4444 - 0.00 seconds
Cgl0004I processed model has 1 rows, 5 columns (5 integer (5 of which binary)) and 5 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.444444
Cbc0038I Solution found of -26
Cbc0038I Before mini branch and bound, 4 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 5 colu

1

In [3]:
# Print the solution
for var in model.variables():
    print(var.name, var.value())

x1 1.0
x2 1.0
x3 0.0
x4 0.0
x5 1.0


Solution: <em>Write the items selected below.</em>

- Items selected: <b> Vest, light meter, baton </b>

# Question 2: Branch And Bound (30 points)
Branch and bound is one of the most common ways to solve optimization problems, including IP problems. It's a search algorithm that iteratively explores the space to find the optimal solution. 

Start by reading through this explanation: https://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf and writing pseudocode for the algorithm below. Then, we'll work through a scaffolded recursive implementation of the algorithm in Python. 

### Pseudocode

### Implementation

We start by defining our arrays for the system (c, A, b). c is an array of the coefficients of the objective function, and A and b are for each constraint. For example: 

$$ Z = 4x_1 + 3 x_2 + x_3$$
$$ 3x_1 + 2x_2 + x_3 <= 7$$
$$ 2x_1 + x_2 + 2x_3 <= 11$$

From here, c = [4, 3, 1], A is [[3, 2, 1], [2, 1, 2]], and b is [7, 11]. 

If there are any upper or lower bounds on any variables, add those to the lb and ub list.


- solve relaxed LP
- if it's infeasible, prune?
- if the optimal solution is integers, return!
- select variable with greater fractional part and create two branches
- one branch with new constraint variable >= variable rounded up
- one branch with new constraint variable <= variable rounded down
- recursibly call function on each branch?

In [25]:
# Similar to the text, we'll have a function to find the relaxed solution of the IP (including decimal points)
import scipy
import numpy as np

"""
Python implementation of branch and bound algorithm for integer programming problems.

Args:
    c: array with objective function coefficients 
    A: 2D array with constraint coefficients
    b: array with constraint values
    lb: array with lower bound values for each variable
    ub: array with upper bound values for each variable

Returns: an array for values of each variable and the optimal objective function value
"""
def branch_and_bound(c, A, b, lb, ub):
    optimized = scipy.optimize.linprog(-c, A, b, method="highs", bounds=list(zip(lb, ub))) # Function to optimize current 

    x = optimized.x
    z = optimized.fun

    # TODO: First check if the problem is feasible or not (check if Z exists).
    if z == None:
        return [], 0
    
    # TODO: Then, check if the current variables are all integers or not. 
    all_integers = True
    for num in x:
        if num % 1 != 0:
            all_integers = False
    
    # TODO: If all variables are integers, we have reached the solution 
    if all_integers:
        return x, -z
    
    # Otherwise we need to branch the value with the most decimal difference similar to how the paper explained
    else:
        # TODO: Start by copying the lower bounds and upper bounds into new arrays. This is how we establish branching
        left_lb = np.copy(lb)
        left_up = np.copy(ub)

        right_lb = np.copy(lb)
        right_up = np.copy(ub)

        # TODO: Find which variable has the maximum decimal difference and save the index (HINT: what math operation deals with remainders?)
        max_difference_idx = None
        max_frac = 0

        for i, num in enumerate(x):
            if num % 1 > max_frac:
                max_difference_idx = i
                max_frac = num % 1

        # TODO: Set the upper and lower bounds based on branching at that variable (floor and ceiling)
        left_up[max_difference_idx] = np.floor(x[max_difference_idx])
        right_lb[max_difference_idx] = np.ceil(x[max_difference_idx])

        # TODO: Recursively get values for the optimized solution of the left and right nodes
        x_left, z_left = branch_and_bound(c, A, b, left_lb, left_up)
        x_right, z_right = branch_and_bound(c, A, b, right_lb, right_up)

        # TODO:Return the coefficients with the higher value of the two (HINT: This takes an if statement)
        if z_left > z_right:
            return x_left, z_left
        return x_right, z_right


Now, lets test the function you've written on the sample problem written above. 

In [26]:
c = np.array([4, 3, 1])
A = np.array([[3, 2, 1], [2, 1, 2]])
b = np.array([7, 11])

lb = np.array([0, 0, 0])
ub = np.array([None, None, None])

x, z = branch_and_bound(c, A, b, lb, ub)

print("Variable")
for idx, var in enumerate(x): 
    print(f">  x{idx} = {var}")

print(f"Objective Function: {z}")

Variable
>  x0 = 1.0
>  x1 = 2.0
>  x2 = 0.0
Objective Function: 10.0


If correct, your solution should be x0 = 1, x1 = 2, and x2 = 0